# Set up analysis

In [128]:
# Load modules
import os
import glob
import datacube
import itertools
import rasterio
import rasterio.features
import fiona
import xarray as xr
import numpy as np
from datacube.storage import masking
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from matplotlib import pyplot as plt
from osgeo import gdal
from shapely.geometry import Point
from shapely.geometry import shape
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import export_graphviz

# Set up datacube instance
dc = datacube.Datacube(app = 'Random forest classification')

def rasterize_vector(input_data, cols, rows, geo_transform,
                     projection, field):
    
    """
    Rasterize a vector file and return numpy array
    :attr vector_data_path: input shapefile path
    :attr cols: width of output array in columns
    :attr rows: height of output array in rows
    :attr geo_transform: geotransform for rasterization
    :attr projection: projection for rasterization
    :attr field: shapefile field to take values from
    :returns: a 'row x col' array containg values from vector
    """
    
    if isinstance(input_data, str):    
    
        # Open vector with gdal
        data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
        input_data = data_source.GetLayer(0)
    
    # Set up output raster
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    
    # Rasterize shapefile and extract array
    gdal.RasterizeLayer(target_ds, [1], input_data, options=["ATTRIBUTE=" + field])
    band = target_ds.GetRasterBand(1)
    out_array = band.ReadAsArray()
    target_ds = None
    
    return out_array


def write_geotiff(fname, data, geo_transform, projection, nodata_val):
    
    """
    Create a single band GeoTIFF file with data from array.
    :attr fname: output file path
    :attr data: input array
    :attr geo_transform: geotransform for output raster
    :attr projection: projection for output raster
    :attr nodata_val: value to convert to nodata in output raster
    """
    
    # Set up driver
    driver = gdal.GetDriverByName('GTiff')
    
    # Create raster of given size and projection
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    
    # Write data to array and set nodata values
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    band.SetNoDataValue(nodata_val)
    
    # Close file
    dataset = None  
    
    
# def write_multibandgeotiff(filename, dataset, time_index=None, profile_override=None):
    
#     """
#     Write an xarray dataset to a geotiff
#     :attr bands: ordered list of dataset names
#     :attr time_index: time index to write to file
#     :attr dataset: xarray dataset containing multiple bands to write to file
#     :attr profile_override: option dict, overrides rasterio file creation options.
#     """
    
#     # Geotiff settings for export
#     DEFAULT_PROFILE = {
#         'blockxsize': 128,  # changed from 256
#         'blockysize': 128,  # changed from 256 
#         'compress': 'lzw',
#         'driver': 'GTiff',
#         'interleave': 'band',
#         'nodata': 9999,
#         'tiled': True}
    
#     profile_override = profile_override or {}

#     dtypes = {val.dtype for val in dataset.data_vars.values()}
#     assert len(dtypes) == 1  # Check for multiple dtypes
#     profile = DEFAULT_PROFILE.copy()
#     profile.update({'width': dataset.dims[dataset.crs.dimensions[1]],
#                     'height': dataset.dims[dataset.crs.dimensions[0]],
#                     'transform': dataset.affine,
#                     'crs': dataset.crs.crs_str,
#                     'count': len(dataset.data_vars),
#                     'dtype': str(dtypes.pop())})
#     profile.update(profile_override)
#     with rasterio.open(filename, 'w', **profile) as dest:
#         for bandnum, data in enumerate(dataset.data_vars.values(), start=1):
#             dest.write(data.data, bandnum)

            
def pq_fuser(dest, src):
    
    """Fuse datacube PQ data"""
    
    valid_val = (1 << valid_bit)

    no_data_dest_mask = ~(dest & valid_val).astype(bool)
    np.copyto(dest, src, where=no_data_dest_mask)

    both_data_mask = (valid_val & dest & src).astype(bool)
    np.copyto(dest, src & dest, where=both_data_mask)

    
def hltc_import(query):
    
    """
    Imports high and low composite data for a given spatial query, and
    return an array and projection information
    :attr query: spatial query for datacube.load()
    :returns: 'row x col x band' array, geo transform and projection string
    """

    # Import data
    low_tide = dc.load(product = 'low_tide_comp_20p', **query)
    high_tide = dc.load(product = 'high_tide_comp_20p', **query)

    # Extract transform/projection from xarray dataset
    geo_transform = low_tide.geobox.transform.to_gdal()
    proj = low_tide.geobox.crs.wkt

    # Rename variables in each high/low composite so datasets can be merged
    data_vars = list(low_tide.keys())[3:]  # select only data vars, not coords
    low_tide.rename({var: "lt_" + var for var in data_vars}, inplace = True)
    high_tide.rename({var: "ht_" + var for var in data_vars}, inplace = True)

    # Combine into one dataset
    raster_input = xr.auto_combine([low_tide, high_tide]).isel(time = 0)

    # Convert to array; reorder dimensions
    bands_array = raster_input.to_array().values
    bands_array = np.einsum('bxy->xyb', bands_array)
    
    return bands_array, geo_transform, proj


def point_in_poly(lon, lat, field, shapefile):
    
    """
    Extract data from polygon that contains point
    :attr lon: x coordinate of point
    :attr lat: y coordinate of point
    :attr field: field of polygon shapefile to extract for matching points
    :attr shapefile: polygon shapefile to import
    :returns: if point falls within polygon, return field from polygon
    """

    # Convert coordinates to shapely point
    mypoint = Point(lon, lat)   

    # Extract polygon info
    with fiona.open(shapefile) as shp:

        # For each polygon, identify if point falls within polygon
        poly_idx = [poly['properties']['ID'] for i, poly in enumerate(shp)
                    if mypoint.within(shape(poly['geometry']))]

    # If point is within polygon
    if poly_idx:
        
        # Take first polygon to avoid multiple matches        
        return(poly_idx[0])

    # If point not found in any polygon, return none
    else: 
        
        return None


def layer_extent(layer):
    
    """
    Computes min and max extents for GDAL layer features. Compared to
    built-in ".GetExtent" that always returns unfiltered extents, this 
    allows you to compute extents of features within filtered layers 
    (e.g. layers filtered with 'SetAttributeFilter')
    """
    
    # Extract tuples of x, y, z coordinates for each point feature
    point_coords = [feature.geometry().GetPoint() for feature in layer]
    
    # Compute mins and maxes across points for each tuple element
    max_x, max_y, max_z = map(min, zip(*point_coords))
    min_x, min_y, min_z = map(max, zip(*point_coords))    
    
    return  min_x, max_x, min_y, max_y


# Set up paths
classification_output = "output_data/classification_dc_hltc_mangroves.tiff"
train_data_path = "raw_data/train"
# validation_data_path = "raw_data/test"


# Import HLTC training data and fit model

In [164]:
# Iterate through each point and extract matching polygon data
with fiona.open(train_data_path + "/training_data_mangrove.shp") as points:
        
        # Output IDs
        poly_ids = list()
        
        # For each point, identify what polygon it belongs to        
        for point in points:
            
            # Convert albers coordinates to WGS84    
            lon, lat = point['geometry']['coordinates']        
            x, y = geometry.point(lon, lat, CRS('EPSG:3577')).to_crs(CRS('WGS84')).points[0]
            
            poly_id = point_in_poly(lon=x, lat=y,
                                    field='ID',
                                    shapefile='/g/data/r78/intertidal/GA_native_tidal_model.shp')
            
            poly_ids.append(poly_id)

# Open vector of training points with gdal
data_source = gdal.OpenEx(train_data_path + "/training_data_mangrove.shp", gdal.OF_VECTOR)
layer = data_source.GetLayer(0)

# Output training label and pixel arrays
training_labels_list = list()
training_samples_list = list()

# For each polygon, extract datacube data using extent of matching points
# and add resulting spectral data and labels to list of arrays
for polygon_id in set(poly_ids):
    
    print("Extracting training data for polygon {}".format(polygon_id))

    # List of matching FIDs
    fid_list = [i for i, x in enumerate(poly_ids) if x == polygon_id]    

    # Temporary fix for polygons containing only one training point: setAttributeFilter
    # returns all polygons if passed a single element tuple, so duplicate the single FID
    if len(fid_list) == 1:  
        fid_list = [fid_list[0], fid_list[0]]     
        
    # Filter layer by passing list of FIDs to SQL query as tuple, i.e. "(1, 2, ...)"
    layer.SetAttributeFilter("FID IN {}".format(tuple(fid_list)))
        
        
    # Compute extents and generate spatial query
    xmin, xmax, ymin, ymax = layer_extent(layer) 
    query_train = {'x': (xmin, xmax),
                   'y': (ymin, ymax),
                   'crs': 'EPSG:3577'}

    # Import data 
    bands_array_train, geo_transform_train, proj_train = hltc_import(query_train)
    rows_train, cols_train, bands_n_train = bands_array_train.shape

    # Import training data shapefiles and convert to matching raster pixels
    training_shapefile = train_data_path + "/training_data_mangrove.shp"
    training_pixels = rasterize_vector(layer, cols_train, rows_train, 
                                       geo_transform_train, proj_train, field = "class")  
    
    # Extract matching image sample data for each labelled pixel location
    is_train = np.nonzero(training_pixels)
    training_labels = training_pixels[is_train]
    training_samples = bands_array_train[is_train]

    # Remove nans from training samples
    training_labels = training_labels[~np.isnan(training_samples).any(axis=1)]
    training_samples = training_samples[~np.isnan(training_samples).any(axis=1)]
    
    # Append outputs
    training_labels_list.append(training_labels)
    training_samples_list.append(training_samples)

# Combine polygon training data    
training_labels = np.concatenate(training_labels_list, axis=0)
training_samples = np.concatenate(training_samples_list, axis=0)  

# Set up classifier and train on training sample data and labels
classifier = RandomForestClassifier(n_jobs = 4, n_estimators = 10)
classifier.fit(training_samples, training_labels)
print("\nModel trained on {} training samples".format(str(len(training_samples))))

# Plot output random forest trees to file
data_vars = [type + var for type in ["lt_", "ht_"] 
             for var in ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']]

# # For each tree in forest
# for n, tree_in_forest in enumerate(classifier.estimators_):

#     # Create graph and save to dot file
#     export_graphviz(tree_in_forest,
#                     out_file="figures/tree_graphs/tree.dot",
#                     feature_names = data_vars,
#                     class_names = ["mangrove", "water", "veg", "other"],
#                     filled=True,
#                     rounded=True)

#     # Plot as figure
#     os.system('dot -Tpng figures/tree_graphs/tree.dot -o ' + \
#               'figures/tree_graphs/tree' + str(n + 1) + '.png')


Extracting training data for polygon 33


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/_collections_abc.py:702: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.
  return len(self._mapping)
/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping


Extracting training data for polygon 6


/g/data/v10/public/modules/agdc-py3/1.5.4/lib/python3.6/site-packages/datacube/utils/__init__.py:180: RuntimeWarning: invalid value encountered in double_scalars
  res = (data[data.size - 1] - data[0]) / (data.size - 1.0)


Extracting training data for polygon 106
Extracting training data for polygon 183
Extracting training data for polygon 157

Model trained on 281 training samples


# Import HLTC analysis data and prepare for classification

In [150]:
# Set up analysis data query
lat_point, lon_point, buffer = -12.5507304443, 130.802900172, 30000
x, y = geometry.point(lon_point, lat_point, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]
query = {'x': (x - buffer, x + buffer),
         'y': (y - buffer, y + buffer),
         'crs': 'EPSG:3577'}

# Import analysis data
bands_array, geo_transform, proj = hltc_import(query)
rows, cols, bands_n = bands_array.shape
print("Data to classify:\nRows: {0}\nColumns: {1}\nBands: {2}".format(rows, cols, bands_n))

# Remove nodata and return flattened 'pixel x bands' array
input_nodata = np.isnan(bands_array).any(axis = 2)
flat_pixels = bands_array[~input_nodata]


/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/_collections_abc.py:702: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.
  return len(self._mapping)
/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/_collections_abc.py:720: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  yield from self._mapping


Data to classify:
Rows: 2401
Columns: 2401
Bands: 12


# Run analysis

In [165]:
# Run classification
print("Classification running...")
result = classifier.predict(flat_pixels)

# Restore 2D array by assigning flattened output to empty array
classification = np.zeros((rows, cols))
classification[~input_nodata] = result
print("  Classification complete")

# Nodata removed
print("  " + str(np.sum(classification == 0)) + " nodata cells removed")

# Export to file
write_geotiff(classification_output, 
              data = classification, 
              geo_transform = geo_transform, 
              projection = proj,
              nodata_val = 0)
print("    Classification exported")


Classification running...
  Classification complete
  0 nodata cells removed
    Classification exported


# Classification statistics

In [45]:
# # Verification data
# shapefiles = glob.glob(validation_data_path + "/*.shp")
# classes = [i.split("/")[2][0:1] for i in shapefiles]
# verification_pixels = vectors_to_raster(shapefiles, rows, cols, geo_transform, proj)
# for_verification = np.nonzero(verification_pixels)
# verification_labels = verification_pixels[for_verification]
# predicted_labels = classification[for_verification]

# # Confusion matrix
# print("Confussion matrix:\n%s" %
#       metrics.confusion_matrix(verification_labels, predicted_labels))
# target_names = ['Class %s' % s for s in classes]

# # Per class report
# print("Classification report:\n%s" %
#       metrics.classification_report(verification_labels, predicted_labels,
#                                     target_names=target_names))

# # Overall classification accuracy
# print("Classification accuracy: %f" %
#       metrics.accuracy_score(verification_labels, predicted_labels))

# Old import NBAR from datacube

In [2]:
# # Define temporal range
# start_of_epoch, end_of_epoch = '2015-01-01', '2015-03-01'

# # Define wavelengths/bands of interest
# bands_of_interest = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']


# # Define sensor of interest
# sensor1 = 'ls8'

# # Location
# lat_point, lon_point, buffer = -31.88, 152.69, 30000
# x, y = geometry.point(lon_point, lat_point, CRS('WGS84')).to_crs(CRS('EPSG:3577')).points[0]

# # Set up query
# query = {'time': (start_of_epoch, end_of_epoch)}
# query['x'] = (x - buffer, x + buffer)
# query['y'] = (y - buffer, y + buffer)
# query['crs'] = 'EPSG:3577'
# print(query)

# # Group PQ by solar day to avoid idiosyncracies of N/S overlap differences in PQ algorithm performance
# pq_albers_product = dc.index.products.get_by_name(sensor1 + '_pq_albers')
# valid_bit = pq_albers_product.measurements['pixelquality']['flags_definition']['contiguous']['bits']

# # Load sensor specific band adjustment tuples for TSS 
# # ls5_tss_constant, ls5_tss_exponent = 3983, 1.6246
# # ls7_tss_constant, ls7_tss_exponent = 3983, 1.6246
# # ls8_tss_constant, ls8_tss_exponent = 3957, 1.6436   

# # Retrieve the NBAR and PQ data for sensor n
# sensor1_nbar = dc.load(product = sensor1 + '_nbar_albers', 
#                        group_by = 'solar_day', 
#                        measurements = bands_of_interest,  
#                        **query)
# sensor1_pq = dc.load(product = sensor1 + '_pq_albers', 
#                      group_by = 'solar_day', 
#                      fuse_func = pq_fuser, 
#                      **query)

# # Extract projection information
# crs = sensor1_nbar.crs.wkt
# affine = sensor1_nbar.affine

# # Ensure 1:1 match between NBAR and PQ
# sensor1_nbar = sensor1_nbar.sel(time = sensor1_pq.time)

# # Generate PQ masks and apply to remove cloud, cloud shadow and saturated observations
# s1_cloud_free = masking.make_mask(sensor1_pq,
#                                   cloud_acca = 'no_cloud',
#                                   cloud_shadow_acca = 'no_cloud_shadow',
#                                   cloud_shadow_fmask = 'no_cloud_shadow',
#                                   cloud_fmask = 'no_cloud',
#                                   blue_saturated = False,
#                                   green_saturated = False,
#                                   red_saturated = False,
#                                   nir_saturated = False,
#                                   swir1_saturated = False,
#                                   swir2_saturated = False,
#                                   contiguous = True)

# # Extract good data
# s1_good_data = s1_cloud_free.pixelquality.loc[start_of_epoch:end_of_epoch]
# sensor1_nbar = sensor1_nbar.where(s1_good_data)
# sensor1_nbar

# # Single clear timestep 6 (time 2015-02-28T23:43:05)
# raster_input = sensor1_nbar.isel(time = 6)
# raster_input.red.plot()

# # Write data to file
# write_multibandgeotiff(filename="raw_data/input_raster.tif", dataset=raster_input) 


{'time': ('2015-01-01', '2015-03-01'), 'x': (1901405.2630724623, 1961405.2630724623), 'y': (-3660643.2668699455, -3600643.2668699455), 'crs': 'EPSG:3577'}


<xarray.Dataset>
Dimensions:  (time: 7, x: 2401, y: 2401)
Coordinates:
  * time     (time) datetime64[ns] 2015-01-11T23:43:21.500000 ...
  * y        (y) float64 -3.601e+06 -3.601e+06 -3.601e+06 -3.601e+06 ...
  * x        (x) float64 1.901e+06 1.901e+06 1.901e+06 1.901e+06 1.902e+06 ...
Data variables:
    blue     (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
    green    (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
    red      (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
    nir      (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
    swir1    (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
    swir2    (time, y, x) float64 nan nan nan nan nan nan nan nan nan nan ...
Attributes:
    crs:      EPSG:3577